# Basic QA for Raya

External Libraries

In [1]:
import matplotlib.pyplot as plt
import cv2
import asyncio
import math
import numpy as np

Ra-Ya

In [2]:
from raya.standalone_handler import StandaloneHandler
from raya.enumerations import ANG_UNIT, POS_UNIT

In [3]:
raya = StandaloneHandler()
await raya.start()

Controllers

In [11]:
lidar = await raya.enable_controller('lidar')
leds = await raya.enable_controller('leds')
interactions = await raya.enable_controller('interactions')
cameras = await raya.enable_controller('cameras')
arms = await raya.enable_controller('arms')
sound = await raya.enable_controller('sound')
motion = await raya.enable_controller('motion')

(WARNING)<RayaApp.RayaApp> Controller "lidar" already enabled.
(WARNING)<RayaApp.RayaApp> Controller "leds" already enabled.
(WARNING)<RayaApp.RayaApp> Controller "interactions" already enabled.
(WARNING)<RayaApp.RayaApp> Controller "cameras" already enabled.
(WARNING)<RayaApp.RayaApp> Controller "arms" already enabled.
(WARNING)<RayaApp.RayaApp> Controller "sound" already enabled.


In [6]:
print(sound.get_predefined_sounds())

['attention', 'very_low_battery', 'malfunction', 'success', 'task_received', 'task_finished', 'error', 'charging']


In [8]:
await sound.play_predefined_sound('task_received', volume=100)

()

# Check motors

## Check rotation

In [12]:
await motion.rotate(angle=90.0, angular_speed=20.0, 
                                 ang_unit=ANG_UNIT.DEG, wait=True)

(False, '')

## Check moving forward

In [ ]:
await motion.move_linear(distance=0.3, x_velocity=0.2, wait=True)


## Check moving backwards

In [ ]:
await motion.move_linear(distance=0.3, x_velocity=-0.2, wait=True)


# Checking Cameras

In [ ]:
try:
    await cameras.enable_color_camera('nav_bottom')
except RayaCameraAlreadyEnabled:
    pass
img = await cameras.get_next_frame('nav_bottom')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
try:
    await cameras.enable_color_camera('nav_top')
except RayaCameraAlreadyEnabled:
    pass
img = await cameras.get_next_frame('nav_top')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Checking Leds

In [ ]:
await leds.animation('head', 'green_general', 'motion_4_in', 3, 5)


In [ ]:
await leds.animation('chest', 'blue_pressed', 'motion_2', 3, 5)

# Checking Lidar

In [ ]:
import copy
def get_lidar_between(lidar_data,
                        lower_angle:float, 
                        upper_angle:float):
        lidar_array = copy.copy(lidar_data)
        step_size = 2*np.pi / len(lidar_array)
        if lower_angle < 0:
            lower_angle = 2 * np.pi + lower_angle 
            upper_angle = 2 * np.pi + upper_angle
        initial_index = math.floor(lower_angle/step_size)
        final_index = math.ceil(upper_angle/step_size)
        lidar_array += lidar_array
        limited_data = lidar_array[initial_index: final_index]
        limited_data = np.flip(limited_data)
        return limited_data

In [ ]:
full_lidar_data = lidar.get_raw_data()

In [ ]:
lower_angle=np.radians(0)
upper_angle=np.radians(360)

In [ ]:
lidar_data_used = get_lidar_between(full_lidar_data, lower_angle, upper_angle)


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='polar')
theta = np.linspace(lower_angle, 
                            upper_angle, 
                            len(lidar_data_used))
        
# Plot
ax1.clear()
ax1.scatter(x=-np.array(theta)-1.578, y=lidar_data_used, s=2)

# Checking Arm

In [ ]:
await arms.set_predefined_pose('right_arm','home', wait=True)

In [ ]:
await arms.set_predefined_pose('right_arm','pre_pick', wait=True)

In [ ]:
await arms.set_predefined_pose('right_arm','pre_step_1', wait=True)

In [ ]:
await arms.set_predefined_pose('right_arm','home', wait=True)